In [84]:
import warnings
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
path = 'C:/Users/6997619/Documents/GitHub projects/DHZW-simulation_Sim-2APL/src/main/resources/sensitivity_analysis/'

## Calculate mean of the iterations of each parameter set 

In [96]:
# DataFrame to store mean values
df_output = pd.DataFrame(columns=["CAR_DRIVER", "CAR_PASSENGER", "BIKE", "BUS_TRAM", "TRAIN", "WALK"])

df_output

,CAR_DRIVER,CAR_PASSENGER,BIKE,BUS_TRAM,TRAIN,WALK


In [97]:
warnings.filterwarnings("ignore")

folder_path = 'C:/Users/6997619/Documents/GitHub projects/DHZW-simulation_Sim-2APL/src/main/resources/sensitivity_analysis/output/iterations'

# Iterate through each CSV file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Calculate the mean of each column
        mean_values = df.mean()
                        
        # Append the mean values to the new DataFrame
        df_output = df_output.append(mean_values, ignore_index=True)

# Enable warnings again
warnings.filterwarnings("default")

In [99]:
df_output

,CAR_DRIVER,CAR_PASSENGER,BIKE,BUS_TRAM,TRAIN,WALK
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
235,NaN,NaN,NaN,NaN,NaN,NaN
236,NaN,NaN,NaN,NaN,NaN,NaN
237,NaN,NaN,NaN,NaN,NaN,NaN
238,NaN,NaN,NaN,NaN,NaN,NaN


## Link means with parameter set for evaluation

In [100]:
df_input = pd.read_csv(path + 'data/parametersets.csv')

#df_output = pd.read_csv(path + 'output/output_proportions.csv')

print(df_output.shape[0])
print(df_input.shape[0])
print(df_output.shape[0] == df_input.shape[0])

# put parameters and results together
df = pd.concat([df_input, df_output], axis=1)

240
240
True


In [32]:
df

,alphaWorkCarDriver,alphaWorkCarPassenger,alphaWorkBus,alphaWorkTrain,betaTimeWorkWalk,betaTimeWorkBike,betaTimeWorkCarDriver,betaTimeWorkCarPassenger,betaTimeWorkBus,betaTimeWorkTrain,...,betaChangesTransportSchool,betaChangesTransportLeisure,used_parameter_label,used_parameter_value,CAR_DRIVER,CAR_PASSENGER,BIKE,BUS_TRAM,TRAIN,WALK
0,0.4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,alphaWorkCarDriver,0.4,NaN,NaN,NaN,NaN,NaN,NaN
1,0.7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,alphaWorkCarDriver,0.7,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,alphaWorkCarDriver,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1.3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,alphaWorkCarDriver,1.3,NaN,NaN,NaN,NaN,NaN,NaN
4,1.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,alphaWorkCarDriver,1.6,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.7,betaChangesTransportLeisure,0.7,NaN,NaN,NaN,NaN,NaN,NaN
237,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,betaChangesTransportLeisure,1.0,NaN,NaN,NaN,NaN,NaN,NaN
238,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.3,betaChangesTransportLeisure,1.3,NaN,NaN,NaN,NaN,NaN,NaN
239,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.6,betaChangesTransportLeisure,1.6,NaN,NaN,NaN,NaN,NaN,NaN


# Plot linear regressions

In [33]:
threshold_slope_abs = 0.005

In [34]:
# Initialize an empty list to store parameter names with slope > 0.1
params_with_slope = []

groups = df.groupby('used_parameter_label')

output_cols = ['WALK', 'BIKE', 'BUS_TRAM', 'TRAIN', 'CAR_DRIVER', 'CAR_PASSENGER']

# Loop over each group and calculate the slope for each parameter
for name, group in groups:
    for col in output_cols:
        # Calculate the slope for this parameter and output column
        slope, intercept = np.polyfit(group['used_parameter_value'], group[col], 1)
        
        # Check if the absolute value of the slope is greater than 0.1
        if abs(slope) > threshold_slope_abs:
            # Add the parameter name to the list
            params_with_slope.append(name)
            break  # Only need to add the parameter name once

# Filter the dataframe to only include rows with parameter names in params_with_slope
df_filtered = df[df['used_parameter_label'].isin(params_with_slope)]

# Group the filtered data by parameter name
groups_filtered = df_filtered.groupby('used_parameter_label')

# Create a new figure with a 3x2 grid of subplots
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(10, 10))

# Initialize a dictionary to store the colors for each group
colors = {}

# Loop over each output column and plot the data in a separate subplot
for i, col in enumerate(output_cols):
    # Get the axis object for this subplot
    ax = axs[i // 2, i % 2]
    
    j = 0
    # Loop over each group and plot the data for this output column
    for name, group in groups_filtered:
        # Extract the x and y values for this group and output column
        x = group['used_parameter_value']
        y = group[col]
        
        # Assign a color to this group based on its name
        if name not in colors:
            colors[name] = plt.cm.tab20(j)
        
        # Plot the data points
        slope, intercept = np.polyfit(x, y, 1)
        regression_line = intercept + slope * x
        ax.plot(x, regression_line, label=name, color=colors[name])
        
        j=j+1
    
    # Set the title for this subplot to the name of the output column
    ax.set_title(col)
    
    # Set the axis labels for this subplot
    ax.set_xlabel('Parameter Value')
    ax.set_ylabel('Mode choice proportion')
    
# Create a legend for the whole figure
handles, labels = [], []
for name, group in groups_filtered:
    for col in output_cols:
        if name in params_with_slope:
            # Check if the parameter name has already been added to the labels list
            if name not in labels:
                # Add a handle to the handles list for this parameter name
                handle = plt.plot([], [], label=name, color=colors[name])[0]
                handles.append(handle)
                labels.append(name)

fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.1), ncol=1, mode='expand')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.3, hspace=0.5)

# Title
fig.suptitle('Parameters linear regression with abs slope coefficient > '+str(threshold_slope_abs), fontsize=16)

# Save
plt.savefig('plot_slopes_'+str(threshold_slope_abs)+'.png', dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

UFuncTypeError: Cannot cast ufunc 'lstsq_n' input 1 from dtype('O') to dtype('float64') with casting rule 'same_kind'

# Rank parameters per mode choice

In [ ]:
all_rankings = pd.DataFrame()

for col in output_cols:
    print(f'Ranking of parameters for {col}:')
    ranking = pd.DataFrame(slopes).loc[col].sort_values(ascending=False)
    print(ranking)
    print('\n')
    ranking.name = col
    all_rankings = pd.concat([all_rankings, ranking], axis=1)

all_rankings.to_csv('parameters_rankings.csv')